## Data pull, parsing and preliminary modeling
#### CS109a - Sam, Luke, Jake, Jeremy
#### Chicago Food Inspection Forecasting
#### Last Updated: 11/28/16

In [ ]:
from download import main

In [ ]:
import datetime
import optparse
import requests
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt
from sys import exit
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
main(date=datetime.datetime.strptime('01/01/2010', '%m/%d/%Y'), export=True)

First we load in the data:

**NOTE: SWAP result binary later**

In [ ]:
import datetime
import optparse
import requests
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt
from sys import exit
import math
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('/Users/jeremywelborn1/Documents/Jeremy/Harvard/Classes/III_Junior/1st_CS109a/DOWNLOADED_DATA.csv')

In [ ]:
result_possibilities = ['Pass', 'Fail', 'Pass w/ Conditions']
valid_result_bools = [result in result_possibilities for result in data.results]
data = data[valid_result_bools]

In [ ]:
def fail(result):
    if 'Fail' in result:
        return 1
    else:
        return 0
    
    
def grocery_store(facility_type):
    if 'Grocery Store' == facility_type:
        return 1
    else:
        return 0

In [ ]:
data['result_binary'] = [fail(result) for result in data.results]
data['grocery'] = [grocery_store(facility_type) for facility_type in data.facility_type]

In [ ]:
top_inspection_types = data.groupby('inspection_type').count().result_binary.sort_values(ascending=False).index[:10]
top_inspection_types_sorted = data.groupby('inspection_type').mean().result_binary[top_inspection_types].sort_values(ascending=False).index

fig = plt.figure() # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.
width = 0.4

data.groupby('inspection_type').mean().result_binary[top_inspection_types].sort_values(ascending=False).plot(kind='bar',color='r',ax = ax, alpha=0.7,width=width,position=0)
data.groupby('inspection_type').count().result_binary[top_inspection_types_sorted].plot(kind='bar',color='b',ax = ax2, alpha=0.7,width=width,position=1)

In [ ]:
inspection_type_dummies = pd.get_dummies(data['inspection_type'])
inspection_type_dummies = inspection_type_dummies[top_inspection_types]


data = pd.concat([data,inspection_type_dummies],axis=1,join='inner')

In [ ]:


biz_cols = []
for col in data.columns:
    if 'business_activity' in col:
        biz_cols.append(col)
        
count = []
mean = []
for col in biz_cols:
    count.append(data[data[col] == 1].result_binary.mean())
    mean.append(len(data[data[col] == 1].result_binary))

In [ ]:
biz_cols_mean = pd.Series(mean,biz_cols)
biz_cols_count = pd.Series(count,biz_cols)

biz_cols_count_top = biz_cols_count.sort_values(ascending=False).index[:10]


biz_col_mean_sorted = biz_cols_mean[biz_cols_count_top].sort_values(ascending=False).index

fig = plt.figure() # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.
width = 0.4

biz_cols_mean[biz_col_mean_sorted].sort_values(ascending=False).plot(kind='bar',color='r',ax = ax, alpha=0.7,width=width,position=0)
biz_cols_count[biz_col_mean_sorted].plot(kind='bar',color='b',ax = ax2, alpha=0.7,width=width,position=1)

In [ ]:

biz_cols_counts = data[biz_cols].sum().sort_values()


pass_rate = []
count = []
for col in biz_cols:
    pass_rate.append(data[data[col] == 1].result_binary.mean())
    pass_rate.append(data[data[col] == 1].result_binary.mean())
    
    
top_n = 20
top_license_counts = biz_cols_counts.ix[biz_cols_counts.index[-top_n:]]

top_biz_counts_cols = list(biz_cols_counts.index[-top_n:])

pass_rate = []
for col in top_biz_counts_cols:
    pass_rate.append(data[data[col] == 1].result_binary.mean())
    
top_biz_pass_rate = pd.Series(data = np.array(pass_rate),index=top_biz_counts_cols)

In [ ]:
pd.DataFrame(1-top_biz_pass_rate).plot(kind='bar')
plt.show()
pd.DataFrame(top_license_counts).plot(kind='bar')
plt.show()

In [ ]:
keep_n=5
all_license_cols = list(biz_cols)
for col in list(biz_cols_counts.index[-keep_n:]):
    all_license_cols.remove(col)

drop_license_cols = all_license_cols
data = data.drop(drop_license_cols,axis=1)

In [ ]:
data['inspection_date'] = pd.to_datetime(data['inspection_date'])
#data['license_start_date'] = pd.to_datetime(data['license_start_date'])
#data['date_issued'] = pd.to_datetime(data['date_issued'])
data['creation_date'] = pd.to_datetime(data['creation_date'])

In [ ]:
# weather_data_ohare = pd.read_csv('Ohare Weather Data.csv')
# weather_data_midway = pd.read_csv('Midway Weather Data.csv')
weather_data_ohare = pd.read_csv('/Users/jeremywelborn1/Documents/Jeremy/Harvard/Classes/III_Junior/1st_CS109a/Ohare Weather Data.csv')
weather_data_midway = pd.read_csv('/Users/jeremywelborn1/Documents/Jeremy/Harvard/Classes/III_Junior/1st_CS109a/Midway Weather Data.csv')

weather_data_ohare['inspection_date'] = [datetime.datetime.strptime(str(DATE), '%Y%m%d') for DATE in weather_data_ohare.DATE]
weather_data_midway['inspection_date'] = [datetime.datetime.strptime(str(DATE), '%Y%m%d') for DATE in weather_data_midway.DATE]
temp_data_ohare = weather_data_ohare.set_index(['inspection_date'])[['TMAX']]
temp_data_midway = weather_data_midway.set_index(['inspection_date'])[['TMAX']]

temp_data_ohare['TMAX_3DayAvg'] = pd.rolling_mean(temp_data_ohare.TMAX,3)
temp_data_ohare['TMAX_10DayAvg'] = pd.rolling_mean(temp_data_ohare.TMAX,10)
temp_data_ohare['TMAX_30DayAvg'] = pd.rolling_mean(temp_data_ohare.TMAX,30)

temp_data_midway['TMAX_3DayAvg'] = pd.rolling_mean(temp_data_midway.TMAX,3)
temp_data_midway['TMAX_10DayAvg'] = pd.rolling_mean(temp_data_midway.TMAX,10)
temp_data_midway['TMAX_30DayAvg'] = pd.rolling_mean(temp_data_midway.TMAX,30)
data = data.join(temp_data_midway, on='inspection_date')

In [ ]:
import re

critical_flags = re.compile(r'(\bCRITICAL\b)',flags = re.IGNORECASE)
serious_flags = re.compile(r'(\bSERIOUS\b)',flags = re.IGNORECASE)
citation_flags = re.compile(r'(\bCITATION\b)',flags = re.IGNORECASE)
violation_flags = re.compile(r'(\bVIOLATION\b)',flags = re.IGNORECASE)
minor_flags = re.compile(r'(\bMINOR\b)',flags = re.IGNORECASE)
corrected_flags = re.compile(r'(\bCORRECTED\b)',flags = re.IGNORECASE)


In [ ]:
violations_count = []
citations_count = []
critical_count = []
serious_count = []
minor_count = []
corrected_count = []

LOG_EVERY_N = 10000


  
for index in data.index:
    text = data.ix[index].violations
    
    if type(text) == type(' '):
        violations_count.append(violation_flags.subn('',text)[1])
        citations_count.append(citation_flags.subn('',text)[1])
        critical_count.append(critical_flags.subn('',text)[1])
        serious_count.append(serious_flags.subn('',text)[1])
        minor_count.append(minor_flags.subn('',text)[1])
        corrected_count.append(corrected_flags.subn('',text)[1])


    else:
        violations_count.append(float('nan'))
        citations_count.append(float('nan'))
        critical_count.append(float('nan'))
        serious_count.append(float('nan'))
        minor_count.append(float('nan'))
        corrected_count.append(float('nan'))

        
    if (index % LOG_EVERY_N) == 0:
        print index


In [ ]:
data['violations_count'] = violations_count
data['citations_count'] = citations_count
data['critical_count'] = critical_count
data['serious_count'] = serious_count
data['minor_count'] = minor_count
data['corrected_count'] = corrected_count

In [ ]:
# compare pass rates for different violation counts
bins = np.linspace(0,6,7)
data[data.result_binary == 1].citations_count.hist(normed=False,alpha=0.5,bins=bins)
data[data.result_binary == 0].citations_count.hist(normed=False,alpha=0.5,bins=bins)

In [ ]:
def prev_inspection_features(data,inspection_id,date):
    current_inspection = data.ix[data[data.inspection_id==inspection_id].index[0]]
    
    all_inspections = data[data.license_ == current_inspection.license_]
    
    previous_inspections = all_inspections[all_inspections.inspection_date < date]
    if len(previous_inspections) >= 1:
        last_inspection_index = previous_inspections.inspection_date.sort_values().index[-1]
        fraction = previous_inspections.result_binary.mean()
        count = previous_inspections.result_binary.count()
        last = previous_inspections.result_binary[last_inspection_index]
        
        #if type(pd.to_datetime('nan')) != type(current_inspection.license_start_date):
        #    license_age = current_inspection.inspection_date - current_inspection.license_start_date
        #    license_age = license_age.days
        #    if license_age <= 0:
        #        license_age = 0
        #else:
        #    license_age = float('nan')
            
        time_since_last = current_inspection.inspection_date - previous_inspections.ix[last_inspection_index].inspection_date
        time_since_last = time_since_last.days
        
        prev_violations = previous_inspections.violations_count[last_inspection_index]
        prev_citations = previous_inspections.citations_count[last_inspection_index]
        prev_critical = previous_inspections.critical_count[last_inspection_index]
        prev_serious = previous_inspections.serious_count[last_inspection_index]
        prev_minor = previous_inspections.minor_count[last_inspection_index]
        prev_corrected = previous_inspections.corrected_count[last_inspection_index]

    else:
        fraction = float('nan')
        count = 0
        last = float('nan')
        license_age = float('nan')
        time_since_last = float('nan')
        prev_violations = float('nan')
        prev_citations = float('nan')
        prev_critical = float('nan')
        prev_serious = float('nan')
        prev_minor = float('nan')
        prev_corrected = float('nan')

    return np.array([count,
                     fraction,
                     last,
                     #license_age,
                     time_since_last,
                     prev_violations,
                     prev_citations,
                     prev_critical,
                     prev_serious,
                     prev_minor,
                     prev_corrected])

In [ ]:
previous_inspection_features = np.zeros((len(data.index),10))

LOG_EVERY_N = 10000
for i in range(len(data.index)):
    index = data.index[i]
    previous_inspection_features[i,:] = prev_inspection_features(data,data.ix[index].inspection_id,data.ix[index].inspection_date)
    if (index % LOG_EVERY_N) == 0:
        print index

In [ ]:
data['previous_count'] = previous_inspection_features[:,0]
data['previous_fraction'] = previous_inspection_features[:,1]
data['previous_result'] = previous_inspection_features[:,2]
#data['license_age'] = previous_inspection_features[:,3]
data['time_since_last_inspection'] = previous_inspection_features[:,3]
data['previous_violations'] = previous_inspection_features[:,4]
data['previous_citations'] = previous_inspection_features[:,5]
data['previous_critical'] = previous_inspection_features[:,6]
data['previous_serious'] = previous_inspection_features[:,7]
data['previous_minor'] = previous_inspection_features[:,8]
data['previous_corrected'] = previous_inspection_features[:,9]

In [ ]:
for col in data.columns:
    print col

In [ ]:
data.to_csv('data_built_features.csv',index=False)

In [ ]:
data.to_csv('BACKUP.csv',index=False)

In [ ]:

#adjust number of buckets using np.linspace ----------------->
yedges = np.linspace(data.longitude.min(),data.longitude.max(),6)
xedges = np.linspace(data.latitude.min(),data.latitude.max(),6)

x = np.array(data.latitude)
y = np.array(data.longitude)

x1 = np.array(data[data.result_binary == 1].latitude)
y1 = np.array(data[data.result_binary == 1].longitude)

x0 = np.array(data[data.result_binary == 0].latitude)
y0 = np.array(data[data.result_binary == 0].longitude)


hist0, xedges, yedges = np.histogram2d(x0, y0, (xedges, yedges))
hist1, xedges, yedges = np.histogram2d(x1, y1, (xedges, yedges))

hist = hist0/(hist1+hist0)
hist_count = hist1+hist0

xidx = np.clip(np.digitize(x, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(y, yedges), 0, hist.shape[1]-1)

c = hist[xidx, yidx]
plt.scatter(x, y, c=c)

plt.show()

sorted_index = np.argsort(hist.reshape(1,-1))

print hist_count.reshape(1,-1)[0][sorted_index[0]]
print hist.reshape(1,-1)[0][sorted_index[0]]

In [ ]:
data['bin_index'] = hist.shape[1]*yidx + xidx

fig = plt.figure(figsize=(10,10)) # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4


data.groupby('bin_index').count().result_binary.plot(kind='bar', color='red', ax=ax, width=width, position=1)
data.groupby('bin_index').mean().result_binary.plot(kind='bar', color='blue', ax=ax2, width=width, position=0)


plt.show()

In [ ]:
result_index = data.groupby('license_description').mean().result_binary.sort_values().index
data.groupby('license_description').mean().result_binary.sort_values()[result_index]

In [ ]:

fig = plt.figure(figsize=(10,10)) # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4


data.groupby('license_description').count().result_binary[result_index].plot(kind='bar', color='red', ax=ax, width=width, position=1)
data.groupby('license_description').mean().result_binary.sort_values()[result_index].plot(kind='bar', color='blue', ax=ax2, width=width, position=0)


plt.show()



In [ ]:
top_50 = data.groupby('facility_type').count().result_binary.sort_values().index[-50:]

In [ ]:
data.groupby('facility_type').mean().result_binary[top_50].sort_values()

top_50count_mean_sorted_index = data.groupby('facility_type').mean().result_binary[top_50].sort_values().index



In [ ]:

fig = plt.figure(figsize=(10,10)) # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4


data.groupby('facility_type').mean().result_binary[top_50].sort_values().plot(kind='bar', color='red', ax=ax, width=width, position=1)
data.groupby('facility_type').count().result_binary[top_50count_mean_sorted_index].plot(kind='bar', color='blue', ax=ax2, width=width, position=0)


plt.show()


## Modeling
#### All shit above is Luke's work

### Get data and desired features

In [ ]:
# data = pd.read_csv('../data_built_features.csv')

In [ ]:
for col in data.columns.values:
    print col

In [ ]:
# we should return to this and check which covariates we've selected 

cols = ['inspection_date',
        'facility_type',
        'latitude',
        'longitude',
        'results',
        'risk',
        'business_activity_Consumption of Liquor on Premises',
        'business_activity_Preparation of Food and Dining on Premise With Seating',
        'business_activity_Retail Sale of Tobacco',
        'business_activity_Retail Sales of Packaged Liquor',
        'business_activity_Retail Sales of Perishable Foods',
        'Canvass', # inspection_type
        'License',
        'Canvass Re-Inspection',
        'Complaint',
        'License Re-Inspection',
        'Short Form Complaint',
        'Complaint Re-Inspection',
        'Suspected Food Poisoning',
        'Consultation',
        'License-Task Force',
        'point_crime_count',
        'point_sanit_count',
        'result_binary',
        'TMAX',
        'TMAX_3DayAvg',
        'TMAX_10DayAvg',
        'TMAX_30DayAvg',
        'previous_count',
        'previous_fraction',
        'previous_result',
        'time_since_last_inspection',
        'previous_violations',
        'previous_citations',
        'previous_critical',
        'previous_serious',
        'previous_minor',
        'previous_corrected']

print data[cols].shape
print data.shape

# select specific features
data_sub = data[cols]
# what would happen if we dropped NaNs
print data_sub.dropna(axis=0, how='any').shape

In [ ]:
data_sub = data_sub.copy()

In [ ]:
# temporary -- dumping correct crime data and sanitation complaint data into dataset
data_dump = pd.read_csv('../data_with_crime_sanit.csv')

In [ ]:
data_sub['crime'] = data_dump['crime'].copy()
data_sub['sanit'] = data_dump['sanitation'].copy()

In [ ]:
# convert string to datetime to be safe
data_sub['inspection_date'] = pd.to_datetime(data_sub['inspection_date'])

print data_sub['inspection_date'].min()
print data_sub['inspection_date'].max()


### Imputation and encoding 

In [ ]:
# non-KNN approach:
# replace null values with feature mean (quantitative) or feature mode (qualitative)
# for column in data_sub.columns:
#     if (data_sub[column].unique().shape[0] < 8) or (data_sub[column].dtype == np.dtype('object')):
#         data_sub[column].fillna(value = data_sub[column].mode()[0], inplace = True)
#     else:
#         data_sub[column].fillna(value = data_sub[column].mean(), inplace = True)

# check no null values
nan_cnt = 0
for i, column in enumerate(data_sub.columns):
    print i, column, np.array(data_sub[column].isnull()).sum()
    nan_cnt = nan_cnt + np.array(data_sub[column].isnull()).sum()
    
print 'total cells:', data_sub.size
print 'nan_cnt:', nan_cnt
print 'ratio for sam:', nan_cnt / float(data_sub.size)



In [ ]:
# how should we handle the historical count data?
# there's no optimal solution: either (1) impute, biasing that they wouldn't have done well 
# or (2) set to 0, biasing that they would've done well? We'll set to 0 for simplicity.

print 'count of obs for which previous_count == 0:', data_sub[data_sub['previous_count'] == 0.0].shape[0]
print 'this corresponds with number of nans for previous_fraction, previous_result, time_since_last_inspection, which is what we\'d expect'

# set all these to 0 then
hist_cols = ['previous_fraction',
            'previous_result',
            'time_since_last_inspection',
            'previous_violations',
            'previous_citations',
            'previous_critical',
            'previous_serious',
            'previous_minor',
            'previous_corrected']

for hist_col in hist_cols:
    data_sub.loc[data_sub['previous_count'] == 0.0, hist_col] = data_sub.loc[data_sub['previous_count'] == 0.0, hist_col].fillna(value = 0)
    
print '\n here are NaN counts after this step: \n'
for i, column in enumerate(data_sub.columns):
    print i, column, np.array(data_sub[column].isnull()).sum()
    
print '37-42 still have a uniform number of null values because they had no text from the inspection, so nothing to scrape'

In [ ]:
# obs with no null values
print 'count of obs with no null values:', data_sub.shape[0] - data_sub.isnull().any(axis = 1).sum()
print 'count of features with no null values:', data_sub.shape[1] - data_sub.isnull().any(axis = 0).sum()

In [ ]:
# features with no null values we can use to impute
data_sub.isnull().any(axis = 0)[data_sub.isnull().any(axis = 0) == False].index.values

In [ ]:
# so that the imputer has enough predictors, to work with, fill in risk by mean / mode approach since only 63 obs have nulls

data_sub['risk'].fillna(value = data_sub['risk'].mode()[0], inplace = True)

# same
# data_sub['inspection_type'].fillna(value = data_sub['inspection_type'].mode()[0], inplace = True)

# check
print 'number of null values in risk:', data_sub['risk'].isnull().sum()
# print 'number of null values in inspection_type:', data_sub['inspection_type'].isnull().sum()

In [ ]:
# KNN 
from sklearn.neighbors import NearestNeighbors

In [ ]:
# have to encode the data first... but facility_type is annoying to do...
print data_sub.facility_type.value_counts()[0:10]

take_types = data_sub.facility_type.value_counts()[0:50].index.values

In [ ]:
# why don't we just take the first fifty for now 
# this takes awhile... a faster approach?
#
# READ THIS IN, DONT BOTHER RUNNING AGAIN
#
for i, row in data_sub[['facility_type']].iterrows():
    if row.values not in take_types:
        data_sub.loc[i, 'facility_type'] = 'Other'
    else:
        continue
    if (i % 500 == 0):
        print 'finished iteration:', i
        
        
# it's encoded in this, so just kidding
# hack_data = data = pd.read_csv('../data_ready.csv')
# hack_data['facility_type']

In [ ]:
# check response
print 'Useless response count:', sum((data_sub.results != 'Pass') & (data_sub.results != 'Pass w/ Conditions') & (data_sub.results != 'Fail'))
indices = (data_sub.results == 'Pass') | (data_sub.results == 'Pass w/ Conditions') | (data_sub.results == 'Fail')
data_sub = data_sub[indices]

In [ ]:
# risk
indices = (data_sub.risk == 'Risk 1 (High)') | (data_sub.risk == 'Risk 2 (Medium)') | (data_sub.risk == 'Risk 3 (Low)')
data_sub = data_sub[indices]

In [ ]:
data_sub = data_sub.reset_index(drop=True)

In [ ]:
# have to these encode first
data_sub_encoded = pd.DataFrame({})
for column in data_sub.columns:
#     if (data_sub[column].unique().shape[0] < 8) or data_sub[column].dtype == np.dtype('object'):
    if (data_sub[column].dtype == np.dtype('object')) & (column not in ['results']):
        encoding = pd.get_dummies(data_sub[column])
        data_sub_encoded = pd.concat([data_sub_encoded, encoding], axis = 1)
    else:
#         data_sub_encoded = pd.concat([data_sub_encoded, data_sub[[column]].astype(float)], axis = 1)
        data_sub_encoded = pd.concat([data_sub_encoded, data_sub[[column]]], axis = 1)

In [ ]:
# train data are observations that are complete
# test data (which we wish to impute) are observations that are incomplete
train = data_sub_encoded[np.logical_not(data_sub_encoded.isnull().any(axis = 1).values)]
test = data_sub_encoded[data_sub_encoded.isnull().any(axis = 1).values]

In [ ]:
# reset indices because we've dropped rows, NearestNeighbors was returning wrong indices because they weren't reset
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
# dont actually have to split x and y since we specify what features we're imputing on in impute_cols
# x_train = train.drop(['result_binary', 'results'], axis = 1) 
# x_test = test.drop(['result_binary', 'results'], axis = 1) 

In [ ]:
data_sub.isnull().any(axis = 0)[data_sub.isnull().any(axis = 0) == False].index.values

In [ ]:
# should be using 'inspection_type', facility_type' too but i will return to this, was frustrated with the encoding!
# these have to be encoded to work with knn, but when encoded it is hard to get them...

impute_cols = ['Risk 1 (High)', 'Risk 2 (Medium)', 'Risk 3 (Low)', 'previous_count', 'previous_fraction',
       'previous_result', 'time_since_last_inspection']

In [ ]:
knn = NearestNeighbors(n_neighbors = 5)
knn.fit(train[impute_cols]) 

In [ ]:
# return k nearest neighbors based on features we have for all observations
# then fill in other values using these nearest neighbors 

import math
from scipy import stats

for i, row in test.iterrows(): # for each observation with incomplete features
    ind = knn.kneighbors(X = row[impute_cols].values.reshape(1,-1), return_distance=False)[0] # return indices of nearest neighbors with complete features
    for col in test.columns.values[row.isnull().values]: # for each feature that're NaNs    
        train_vals = np.array(train.loc[ind, col]) # get vals from nearest neighbors for this col
        if train[col].unique().shape[0] <= 2: # if indicator 
            test.loc[i, col] = stats.mode(train_vals)[0][0] # fill w/ mode
        else:
            test.loc[i, col] = train_vals.mean() # fill w/ mean
    if (i % 500 == 0):
        print 'finished iteration:', i

In [ ]:
# this won't work, apply flattens the thing
# import math
# from scipy import stats

# def impute_row(row):
#     ind = knn.kneighbors(X = row[impute_cols].values.reshape(1, -1), return_distance=False)[0]
#     for col in test.columns.values[row.isnull().values]: # for each feature that're NaNs    
#         train_vals = np.array(train.loc[ind, col])
#         if train[col].unique().shape[0] <= 2:
#             row[col] = stats.mode(train_vals)[0][0]
#         else:
#             row[col] = train_vals.mean()

# test = test.apply(impute_row, axis = 1)


In [ ]:
data_ready = pd.concat([train, test], axis = 0)
data_ready = data_ready.reset_index(drop=True)

In [ ]:
# reorder = ['inspection_date','result_binary','results','point_crime_count', 'point_sanit_count',
#       'latitude', 'longitude','TMAX','TMAX_3DayAvg', 'TMAX_10DayAvg', 'TMAX_30DayAvg',
#      'Risk 1 (High)', 'Risk 2 (Medium)', 'Risk 3 (Low)',
#         'previous_count','previous_fraction', 'previous_result',
#        'time_since_last_inspection', 'previous_violations',
#        'previous_citations', 'previous_critical', 'previous_serious',
#        'previous_minor', 'previous_corrected','inspection_type',
#         "1023 CHILDERN'S SERVICES FACILITY",
#        "1023-CHILDREN'S SERVICES FACILITY", 'ASSISTED LIVING', 'BANQUET',
#        'BANQUET HALL', 'Bakery', 'CAFETERIA', 'CHURCH', 'CHURCH KITCHEN',
#        'CONVENIENCE', 'CONVENIENCE STORE', 'CULINARY SCHOOL', 'Catering',
#        "Children's Services Facility", 'DAYCARE', 'Daycare (2 - 6 Years)',
#        'Daycare (2 Years)', 'Daycare (Under 2 Years)',
#        'Daycare Above and Under 2 Years', 'Daycare Combo 1586',
#        'GAS STATION', 'GAS STATION/MINI MART', 'GROCERY/RESTAURANT',
#        'Golden Diner', 'Grocery Store', 'Hospital', 'KIOSK',
#        'LIVE POULTRY', 'Liquor', 'Long Term Care', 'Mobile Food Dispenser',
#        'Mobile Food Preparer', 'Navy Pier Kiosk', 'Other',
#        'PRIVATE SCHOOL', 'RESTAURANT/BAR', 'RESTAURANT/GROCERY STORE',
#        'ROOF TOPS', 'ROOFTOP', 'Restaurant', 'STADIUM', 'STORE', 'School',
#        'Shared Kitchen', 'Shared Kitchen User (Long Term)',
#        'Shared Kitchen User (Short Term)', 'Shelter', 'Special Event',
#        'TAVERN', 'Wholesale', 'convenience store', 
#        'business_activity_Catering of Liquor To Events',
#        'business_activity_Consumption of Liquor on Premises',
#        'business_activity_Hotel - 7 or More Sleeping Rooms',
#        'business_activity_Operation of a Fuel Filling Station',
#        'business_activity_Preparation of Food and Dining on Premise With Seating',
#        'business_activity_Provides Onsite Amusement or Entertainment',
#        'business_activity_Retail Sale of Tobacco',
#        'business_activity_Retail Sales of Packaged Liquor',
#        'business_activity_Retail Sales of Packaged Liquor on Sundays from 8AM - 11AM | Retail Sales of Packaged Liquor',
#        'business_activity_Retail Sales of Packaged Liquor | Retail Sales of Packaged Liquor on Sundays from 8AM - 11AM',
#        'business_activity_Retail Sales of Perishable Foods',
#        'business_activity_Retail Sales of Tobacco Products',
#        'business_activity_Sale of Food Prepared Onsite With Dining Area',
#        'business_activity_Sale of Food Prepared Onsite Without Dining Area',
#        'business_activity_Sale of Liquor Outdoors on Private Property',
#        'business_activity_Sale of Liquor Until 4am, Monday - Saturday and 5am on Sunday',
#        'business_activity_Supervision of, and Care for, Children 0-6 Years of Age, During the Day Between 6am-9pm',
#        'business_activity_Supervision of, and Care for, Children 2-6 Years of Age, During the Day Between 6am-9pm',
#        'business_activity_Supervision of, and Care for, Children Under 2 Years of Age, During the Day Between 6am-9pm | Supervision of, and Care for, Children 2-6 Years of Age, During the Day Between 6am-9pm',
#        'business_activity_Tavern - Consumption of Liquor on Premise']

# data_ready = data_ready[reorder]

In [ ]:
# so don't have to rerun the code above
data_ready.to_csv('/Users/jeremywelborn1/Documents/Jeremy/Harvard/Classes/III_Junior/1st_CS109a/fggw/data_ready.csv')

#### Scoring function 
**How we select a model and how this reflects the model put into practice:**  
Ultimately, we care only that we correctly identify failed inspections among the inspections that the City of Chicago actually has the resources to carry out. So, we'd like our model to produce probabilities of passing or not passing -- the inspectors can then inspect establishments with highest probailities of not passing. We'd also like to punish or penalize the model for sending inspectors to inspect establishments that pass but that we predict not to pass with high certainty (or vice versa) (i.e. we'd like to be less wrong when we're not right). 

The log loss is a suitable objective function to optimize then. The log loss is the negative log likelihood of a Bernoulli random variable (in the 2-class setting, we'll justify this shortly): $$-\frac{1}{n}\sum_1^n y_i \log(p_i)-(1-y_i)\log(1-p_i)$$ for $n$ observations, where the $i$th observation is of correct class $y_i \in \{0,1\}$ which our model predicts with probability $p_i$. This achieves specifically this sort of penality (suppose $y_i=1$ and we predict this with probability of only $0.1$, this yields a value that approaches $-\infty$ rapidly).

When put into practice, this approach ranks inspections by probability of not passing, so inspectors can carry out inspections that appear at the top of this ranking. Selecting a model with minimal log loss (or maximum likelihood) essentially ensures this ranking is best, that inspectors have the highest chance of inspecting establishments that have commited a violation. Ultimately, in a given time frame with constrained resources, the City of Chicago cannot carry out all the inspections it has to do. There is a cutoff, so they will carry out the inspections at the top of this ranking. 

How can we get confidence in this approach (what does it look like in practice)? Suppose this model is run only one time (that's the best we can do, we don't have anymore data) and we use the resulting ranking to allocate our inspectors to inspections for $n$ inspections (we take $n$ to be the number of inspections the inspectors were able to do in, say, a month). We then compare the number of failures correctly classified in the top $n$ inspections in the ranking (since there are what would've been done using our best model and methodology) to the number of failures actually found in this same timeframe. If it is greater, then we should be confident in how the model is allocating the City of Chicago's resources. (N.B. of course, the same number of failures will all be discovered over time, but we care about early intervention from a public health perspective!).

Also note that a 2-class approach is sufficient here - we care about failing (vs. not failing). 'Pass' vs. 'Pass w/ Conditions' is an unimportant distinction for how this model is motivated.

In [ ]:
import scipy as sp

# true is list of true classes, pred is list of predicted class probabilities 
def score(y, p_hat):
    p_hat = map(lambda t: t[1], p_hat) # p_hat returned from sklearn is a list of lists with p for both classes, we want p for class 1 or fail 
    epsilon = 1e-15
    p_hat = sp.maximum(epsilon, p_hat)
    p_hat = sp.minimum(1-epsilon, p_hat)
    logloss = sum(y*sp.log(p_hat) + sp.subtract(1,y)*sp.log(sp.subtract(1,p_hat)))
    logloss = logloss * -1.0/len(y)
    return logloss

# def score(act, pred, n):
#     act = act[:n]
#     pred = pred(:n)
#     epsilon = 1e-15
#     pred = sp.maximum(epsilon, pred)
#     pred = sp.minimum(1-epsilon, pred)
#     ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
#     ll = ll * -1.0/len(act)
#     return ll

#### Make models
Logistic
Poly logistic
LDA
QDA
k-NN
Trees (bagging RF, boosting AdaBoost)
SVM

In [1]:
import datetime
import optparse
import requests
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt
from sys import exit
import math
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest 
from sklearn.ensemble import AdaBoostClassifier as AdaBoost
from sklearn.svm import SVC

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
dataset = pd.read_csv('/Users/jeremywelborn1/Documents/Jeremy/Harvard/Classes/III_Junior/1st_CS109a/fggw/data_ready.csv')
# dataset = data_ready

In [4]:
# convert to access the year conveniently
dataset['inspection_date'] = pd.to_datetime(dataset['inspection_date'])
dataset['inspection_date'] = [day.date() for day in dataset['inspection_date']]

In [5]:
print dataset['inspection_date'].min()
print dataset['inspection_date'].max()


2010-01-04
2016-12-02


In [6]:
# DONT RUN THIS, Luke adjusted 0, 1 labeling up above
# swap 0s and 1s to make neg log loss scoring functions more easy to interpret (we care about fails)
# dataset.replace(to_replace = {'result_binary': {0: 'not pass', 1: 'pass'}}, inplace = True)
# dataset.replace(to_replace = {'result_binary': {'not pass': 1, 'pass': 0}}, inplace = True)

In [7]:
# confirm this 
# dataset.loc[0:10, ['result_binary', 'results']]

In [8]:
np.unique(np.array(map(lambda d: d.year, dataset['inspection_date'].values)))

array([2010, 2011, 2012, 2013, 2014, 2015, 2016])

In [10]:
# withold 2010 for lookback period to build previous features
# train on 2011-2015
train = dataset[map(lambda d: (d.year != 2010) & (d.year != 2016), dataset['inspection_date'].values)] 
# test on 2016 - not this notebook, this is Luke
test = dataset[map(lambda d: d.year == 2016, dataset['inspection_date'].values)] 

x_train = train.drop(['result_binary', 'results', 'inspection_date'], axis = 1) # do we want inspection_date anymore? annoying with sklearn functions...
y_train = train['result_binary']
x_test = test.drop(['result_binary', 'results', 'inspection_date'], axis = 1) 
y_test = test['result_binary']

In [11]:
print dataset.shape, train.shape, test.shape
print 'n.b.: not including observations from 2010'

(120304, 93) (87180, 93) (15742, 93)
n.b.: not including observations from 2010


In [12]:
# correct class imbalance in the train data
print 'Count pass:', sum(y_train == 0)
print 'Count not pass:', sum(y_train == 1)

Count pass: 68510
Count not pass: 18670


In [13]:
from sklearn import preprocessing

In [14]:
preprocessing.scale(x_train['crime'], copy = False)
preprocessing.scale(x_train['sanit'], copy = False)
preprocessing.scale(x_test['crime'], copy = False)
preprocessing.scale(x_test['sanit'], copy = False)

array([ 0.18646092,  2.19330602,  0.19956804, ...,  0.22694735,
        0.22694735, -0.28743414])

In [15]:
## subsampling algorithm 
# takes x_train, y_train and returns a class-balanced x_train, y_train 
# assumes count of class 0 > count of class 1
# subsampled down so a bunch of data disappears - we can change this if we want 
def subsample(x_train, y_train):
    train = pd.concat([x_train, y_train], axis = 1)

    train_0 = train[y_train == 0]
    train_1 = train[y_train == 1]
    
    train_0_subsample = train_0.sample(train_1.shape[0])
    
    train_subsample = pd.concat([train_0_subsample, train_1], axis = 0)
    
    x_train_subsample = train_subsample.iloc[:, :-1]
    y_train_subsample = train_subsample.iloc[:, -1]
    
    return x_train_subsample, y_train_subsample
    
x_train_sub, y_train_sub = subsample(x_train, y_train)

In [16]:
print 'Count pass:', sum(y_train_sub == 0)
print 'Count not pass:', sum(y_train_sub == 1)

Count pass: 18670
Count not pass: 18670


In [17]:
x_train = x_train_sub
y_train = y_train_sub

#### Models

In [ ]:
# can't GridSearchCV because I want the predicted probabilities for each combination of hyperparameters,
# GridSearchCV won't give this to me
# so apologies for the copy and pasted code
# i wrote naive_model to return the predicted fail probabilities so Luke can throw them togther in his score function


# approach: train on 2011-2014, validate on 2015 using log loss to select best model per class, test on 2016 to select best model

#### Training a bunch of models for visualization without selecting best model on log loss basis -- skip for site

In [ ]:
p_hats = {}
# here's how im gonna store this shit
# p_hats['model'] = [0.1,0.11,0.2,0.22]
# output = pd.DataFrame(p_hats)
# output.set_index(test['inspection_date'])
# output.to_csv('pred_prob_for_fails.csv',index=False)

In [18]:
def naive_model(clf_instance):
    clf_instance.fit(x_train, y_train)
    print 'accuracy:', round(clf_instance.score(x_test, y_test), 2)
    p_hat = clf_instance.predict_proba(x_test)

    print '\n', 'Confusion matrix:'
    conf = confusion_matrix(y_test, clf_instance.predict(x_test))
    conf = conf / float(conf.sum())
    print conf

    print 'log loss:', round(score(y_test, p_hat), 2)
    return map(lambda t: t[1], p_hat) # p_hat returned from sklearn is a list of lists with p for both classes, we want p for class 1 or fail 

In [ ]:
# logreg
logreg = LogReg(C = 1.0, class_weight = 'balanced') 
naive_model(logreg)

In [ ]:
alphas = map(lambda x: 10**x, np.linspace(-2, 2, 5))
logreg_hyperparams = {'C': alphas}  
logreg_labels = ['LogReg, C = 0.10','LogReg, C = 0.1','LogReg, C = 1','LogReg, C = 10','LogReg, C = 100']

In [ ]:
for i, alpha in enumerate(alphas):
    logreg = LogReg(C = alpha, class_weight = 'balanced') 
    p_hats[logreg_labels[i]] = naive_model(logreg)

In [ ]:
# lda
lda = LDA() 
naive_model(lda)

In [ ]:
shrinkage_params = np.linspace(0.0, 1.0, 5)
# lda_hyperparams = {'shrinkage': shrinkage}  
lda_labels = ['LDA, shrinkage = 0.0','LDA, shrinkage = 0.25','LDA, shrinkage = 0.50','LDA, shrinkage = 0.75','LDA, shrinkage = 1.0']

In [ ]:
for i, shrinkage_param in enumerate(shrinkage_params):
    lda = LDA(shrinkage = shrinkage_param, solver = 'lsqr') 
    p_hats[lda_labels[i]] = naive_model(lda)

In [ ]:
# qda
qda = QDA() 
naive_model(qda)

In [ ]:
reg_params = np.linspace(0.0, 1.0, 5)
qda_labels = ['QDA, reg = 0.0','QDA, reg = 0.25','QDA, reg = 0.50','QDA, reg = 0.75','QDA, reg = 1.0']

In [ ]:
for i, reg_param in enumerate(reg_params):
    qda = QDA(reg_param = reg_param) 
    p_hats[qda_labels[i]] = naive_model(qda)

In [ ]:
# knn
knn = KNN(n_neighbors = 5)
naive_model(knn)

In [ ]:
ns = [5,10,25,50]
knn_labels = ['KNN, n = 5','KNN, n = 10','KNN, n = 25','KNN, n = 50']

In [ ]:
for i, n in enumerate(ns):
    knn = KNN(n_neighbors = n) 
    p_hats[knn_labels[i]] = naive_model(knn)

#### bagging - rf

In [ ]:
# bagging - rf
# non-cross validated
rf = RandomForest()
naive_model(rf)

In [ ]:
x_train.shape[1] / 2

In [ ]:
# cross validated
rf_hyperparams = {'n_estimators': [25,50],
                  'max_features': [x_train.shape[1] / 2, x_train.shape[1]], 
                  'max_depth': [2, 4]}  


In [ ]:
rfs = [(25,x_train.shape[1] / 2),
       (25,x_train.shape[1]), 
       (50,x_train.shape[1] / 2), 
       (50,x_train.shape[1])]

rf_labels = ['RF, n = 25, features = 45','RF, n = 25, features = 90','RF, n = 50, features = 45','RF, n = 50, features = 90']

In [ ]:
for i, rf in enumerate(rfs):
    rf = RandomForest(n_estimators = rf[0], max_features = rf[1]) 
    p_hats[rf_labels[i]] = naive_model(rf)

#### boosting - adaboost

In [ ]:
# boosting - adaboost
# non-cross validated
ada = AdaBoost()
naive_model(ada)

In [ ]:
adas = [25,50,100]

ada_labels = ['AdaBoost, n = 25','AdaBoost, n = 50','AdaBoost, n = 100',]

In [ ]:
for i, ada in enumerate(adas):
    adaboost = RandomForest(n_estimators = ada) 
    p_hats[ada_labels[i]] = naive_model(adaboost)

In [ ]:
p_hats['inspection_date'] = test['inspection_date']
p_hats['result_binary'] = test['result_binary']

In [ ]:
# save this down
output = pd.DataFrame(p_hats)

In [ ]:
output.to_csv('/Users/jeremywelborn1/Documents/Jeremy/Harvard/Classes/III_Junior/1st_CS109a/fggw/pred_prob_for_fails.csv')

#### svm

In [20]:
# svm 
# non-cross validated
# svc = SVC(probability = True)
# pred_prob_svc = naive_model(svc)

In [ ]:
#SLOW
# grid CV
Cs = map(lambda t: 10**t, range(0, 4))
kernels = ['linear', 'poly', 'rbf']
svc_hyperparams = {'C': Cs, 'kernel': kernels}

svc = SVC()
gs_svc = GridSearchCV(svc, param_grid = svc_hyperparams)
gs_svc.fit(x_train, y_train)
p_hat = gs_clf.best_estimator_.predict_proba(x_test)

print score(y_test, p_hat)

#### Training a bunch of models and selecting best model on log loss basis -- for final output

#### Set hyperparameter space

In [134]:
alphas = map(lambda x: 10**x, np.linspace(-4, 4, 9))
logreg_hyperparams = {'C': alphas}  

shrinkage_params = np.linspace(0.0, 1.0, 5)
lda_hyperparams = {'shrinkage': shrinkage_params, 'solver': ['lsqr']}  

reg_params = np.linspace(0.0, 1.0, 5)
qda_hyperparams = {'reg_param': reg_params}

knn_hyperparams = {'n_neighbors': [5,10,25,50]}

rf_hyperparams = {'n_estimators': [25,50],
                  'max_features': [x_train.shape[1] / 2, x_train.shape[1]], 
                  'max_depth': [2, 4]}  

ada_hyperparams = {'n_estimators': [25,50,100]} 

model_classes = [LogReg, LDA, QDA, KNN, RandomForest, AdaBoost]
model_classes_str = ['LogReg', 'LDA', 'QDA', 'KNN', 'RandomForest', 'AdaBoost']
model_class_hyperparams = [logreg_hyperparams, lda_hyperparams, qda_hyperparams, knn_hyperparams, rf_hyperparams, ada_hyperparams]


#### Return best predictions on the test set

In [171]:
p_hats_cv = {} # contains predicted probabilities for each of best models per model class 
clfs = [] # contains best model per model class

for i, model_class in enumerate(model_classes):
    gs_clf = GridSearchCV(model_class(), param_grid = model_class_hyperparams[i], scoring = 'neg_log_loss', cv = 5)
    gs_clf.fit(x_train, y_train)
    best_clf = gs_clf.best_estimator_
    
    score = 'log loss: ' + str(round(-1 * gs_clf.best_score_, 2))
    
    # best predictor label - class, score, and the tuned hyperparameters of interest 
    best_pred_label = model_classes_str[i] + ', ' + score + ', params = ' + \
                    repr([key + ': ' + str(gs_clf.best_estimator_.get_params()[key]) for key in model_class_hyperparams[i].keys()])
    
    # best predictor's predicted probabilities for class 1 fail 
    best_pred_prob = map(lambda t: t[1], best_clf.predict_proba(x_test))    
    
    # store best predictor's probabilities 
    p_hats_cv[best_pred_label] = best_pred_prob
    
    # store classifiers as well
    clfs.append(best_clf)

In [172]:
print 'Grid search selected models:'
p_hats_cv.keys()

Grid search selected models:


["KNN, log loss: 17.24, params = ['n_neighbors: 50']",
 "RandomForest, log loss: 0.94, params = ['n_estimators: 50', 'max_features: 45', 'max_depth: 2']",
 "QDA, log loss: 1.07, params = ['reg_param: 0.25']",
 "AdaBoost, log loss: 0.73, params = ['n_estimators: 100']",
 "LogReg, log loss: 0.75, params = ['C: 1.0']",
 "LDA, log loss: 0.72, params = ['shrinkage: 0.0', 'solver: lsqr']"]

In [139]:
p_hats_cv['inspection_date'] = test['inspection_date']
p_hats_cv['result_binary'] = test['result_binary']
output = pd.DataFrame(p_hats_cv)
output.to_csv('/Users/jeremywelborn1/Documents/Jeremy/Harvard/Classes/III_Junior/1st_CS109a/fggw/pred_prob_for_fails_cv.csv')

In [141]:
# test with this 
# gs_clf = GridSearchCV(LogReg(), param_grid = logreg_hyperparams, scoring = 'neg_log_loss', cv = 5)
# gs_clf.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.0001, 0.001, 0.01, 0.10000000000000001, 1.0, 10.0, 100.0, 1000.0, 10000.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring='neg_log_loss',
       verbose=0)

In [150]:
# use to get best n classifiers rather than best classifier 
#
# takes a fit grid searched classifier and returns best n classifiers' hyperparameters
# def best_params(gs_clf, n): 
#     best_ids = np.array(map(lambda t: t[1], np.array(gs_clf.grid_scores_))).argsort()[::-1][:n]
#     best_params = [gs_clf.grid_scores_[best_id][0] for best_id in best_ids]
#     return best_params